# 🔧 Fed-Audit-GAN v2.0 - CIFAR-10 (Multi-Gamma Ablation Study) - TPU Edition

## 🎯 Experiments Run:
- **FedAvg** (γ = 0.0) - Baseline with data-weighted aggregation (NO GAN)
- **Fed-Audit-GAN γ = 0.3** - Mild fairness weighting
- **Fed-Audit-GAN γ = 0.7** - Strong fairness weighting

## 🔧 4-Phase Architecture:
1. **Phase 1**: Local Client Training (FedProx)
2. **Phase 2**: GAN Training (Fairness Generator)
3. **Phase 3**: Fairness Scoring (with Momentum/EMA)
4. **Phase 4**: Aggregation (with Warm-up)

## 📊 Fairness Metrics (Per-Client Accuracy Based!):
- **JFI**: Jain's Fairness Index
- **Max-Min Fairness**: min(acc)/max(acc)
- **Accuracy Gap**: max(acc) - min(acc)

## 🚀 TPU Optimizations (Kaggle TPU v3-8):
- PyTorch XLA for TPU acceleration
- 8-core TPU with parallel data loading
- Batch size scaled for TPU (128 per core)

---

In [ ]:
# Step 1: Install Dependencies (TPU Edition)
# Install PyTorch XLA for TPU support
!pip install -q torch torchvision tqdm matplotlib numpy wandb
!pip install -q cloud-tpu-client==0.10 torch==2.0.0 torchvision==0.15.1 https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-2.0-cp310-cp310-linux_x86_64.whl

print("✅ Dependencies installed (including PyTorch XLA for TPU)!")

In [ ]:
# Step 2: Login to WandB
import wandb
wandb.login()
print("✅ WandB logged in!")

In [ ]:
# Step 3: Imports and TPU Setup
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
import numpy as np
import copy
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# ============================================================
# 🚀 TPU SETUP (Kaggle TPU v3-8)
# ============================================================

# Check for TPU availability
try:
    import torch_xla
    import torch_xla.core.xla_model as xm
    import torch_xla.distributed.parallel_loader as pl
    import torch_xla.distributed.xla_multiprocessing as xmp
    
    TPU_AVAILABLE = True
    DEVICE = xm.xla_device()
    NUM_TPU_CORES = 8  # TPU v3-8 has 8 cores
    
    print("✅ TPU detected! Using PyTorch XLA")
    print(f"   TPU Device: {DEVICE}")
    print(f"   TPU Cores: {NUM_TPU_CORES}")
    
except ImportError:
    TPU_AVAILABLE = False
    print("⚠️  PyTorch XLA not available. Checking for GPU...")
    
    if torch.cuda.is_available():
        DEVICE = torch.device('cuda')
        print(f"✅ GPU detected: {torch.cuda.get_device_name(0)}")
        torch.backends.cudnn.benchmark = True
        torch.backends.cudnn.enabled = True
    else:
        DEVICE = torch.device('cpu')
        print("⚠️  No accelerator detected. Using CPU (training will be slow).")

# Note: We disable AMP for TPU (XLA handles optimization internally)
USE_AMP = not TPU_AVAILABLE and torch.cuda.is_available()

print(f"\n📍 Device: {DEVICE}")
print(f"   PyTorch: {torch.__version__}")
print(f"   TPU Mode: {TPU_AVAILABLE}")
print(f"   AMP Enabled: {USE_AMP}")

In [ ]:
# ============================================================
# MODEL DEFINITIONS (CIFAR-10 - Deeper CNN)
# ============================================================

class CNN(nn.Module):
    """Deeper CNN for CIFAR-10 classification"""
    def __init__(self, num_classes=10):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.conv4 = nn.Conv2d(256, 512, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(512)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(512 * 2 * 2, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))  # 32->16
        x = self.pool(F.relu(self.bn2(self.conv2(x))))  # 16->8
        x = self.pool(F.relu(self.bn3(self.conv3(x))))  # 8->4
        x = self.pool(F.relu(self.bn4(self.conv4(x))))  # 4->2
        x = x.view(-1, 512 * 2 * 2)
        x = self.dropout(F.relu(self.fc1(x)))
        return self.fc2(x)


class FairnessGenerator(nn.Module):
    """Generator that produces paired samples (x, x') for fairness testing"""
    def __init__(self, latent_dim=128, num_classes=10, img_shape=(3, 32, 32)):
        super().__init__()
        self.latent_dim = latent_dim
        self.num_classes = num_classes
        self.img_shape = img_shape
        self.label_emb = nn.Embedding(num_classes, latent_dim)
        self.init_size = img_shape[1] // 4  # 8
        self.l1 = nn.Linear(latent_dim * 2, 512 * self.init_size ** 2)
        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(512),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(512, 256, 3, 1, 1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(256, 128, 3, 1, 1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),
            nn.Conv2d(128, img_shape[0], 3, 1, 1),
            nn.Tanh()
        )
        self.delta_net = nn.Sequential(
            nn.Linear(latent_dim, 512),
            nn.ReLU(),
            nn.Linear(512, int(np.prod(img_shape))),
            nn.Tanh()
        )
        self.delta_scale = 0.1

    def forward(self, z, labels):
        gen_input = torch.cat([z, self.label_emb(labels)], dim=1)
        out = self.l1(gen_input).view(-1, 512, self.init_size, self.init_size)
        x = self.conv_blocks(out)
        delta = self.delta_net(z).view(-1, *self.img_shape) * self.delta_scale
        return x, torch.clamp(x + delta, -1, 1)


class Discriminator(nn.Module):
    """Conditional Discriminator - outputs logits for BCEWithLogitsLoss"""
    def __init__(self, num_classes=10, img_shape=(3, 32, 32)):
        super().__init__()
        self.num_classes = num_classes
        self.img_shape = img_shape
        self.label_emb = nn.Embedding(num_classes, num_classes)
        self.conv = nn.Sequential(
            nn.Conv2d(img_shape[0] + num_classes, 64, 3, 2, 1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(64, 128, 3, 2, 1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),
            nn.Conv2d(128, 256, 3, 2, 1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2),
            nn.Conv2d(256, 512, 3, 2, 1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2)
        )
        self.fc = nn.Linear(512 * 2 * 2, 1)  # No Sigmoid

    def forward(self, img, labels):
        label_map = self.label_emb(labels).view(-1, self.num_classes, 1, 1)
        label_map = label_map.expand(-1, -1, self.img_shape[1], self.img_shape[2])
        out = self.conv(torch.cat([img, label_map], dim=1))
        return self.fc(out.view(out.size(0), -1))


print("✅ Models defined: CNN, FairnessGenerator, Discriminator")

In [ ]:
# ============================================================
# HELPER FUNCTIONS (TPU/GPU Compatible)
# ============================================================

def train_gan(G, D, model, loader, epochs=20, device=None, l1=1.0, l2=1.0):
    """Train the Fairness GAN (TPU/GPU compatible)"""
    if device is None:
        device = DEVICE
    G, D, model = G.to(device), D.to(device), model.to(device)
    model.eval()
    opt_G = optim.Adam(G.parameters(), lr=0.0002, betas=(0.5, 0.999))
    opt_D = optim.Adam(D.parameters(), lr=0.0002, betas=(0.5, 0.999))
    bce = nn.BCEWithLogitsLoss()
    
    # AMP only for CUDA (not for TPU - XLA handles optimization)
    use_amp_local = USE_AMP and not TPU_AVAILABLE and 'cuda' in str(device)
    if use_amp_local:
        scaler_G = torch.amp.GradScaler(device='cuda')
        scaler_D = torch.amp.GradScaler(device='cuda')
    
    for _ in range(epochs):
        for imgs, labels in loader:
            bs = imgs.size(0)
            real = torch.ones(bs, 1, device=device)
            fake_t = torch.zeros(bs, 1, device=device)
            imgs, labels = imgs.to(device), labels.to(device)
            z = torch.randn(bs, G.latent_dim, device=device)
            gl = torch.randint(0, G.num_classes, (bs,), device=device)
            
            # Generator
            opt_G.zero_grad(set_to_none=True)
            if use_amp_local:
                with torch.amp.autocast(device_type='cuda'):
                    x, xp = G(z, gl)
                    with torch.no_grad():
                        px, pxp = model(x), model(xp)
                    t1 = -torch.mean((px - pxp) ** 2)
                    t2 = l1 * torch.mean((x - xp) ** 2)
                    t3 = l2 * (bce(D(x, gl), real) + bce(D(xp, gl), real)) / 2
                    g_loss = t1 + t2 + t3
                scaler_G.scale(g_loss).backward()
                scaler_G.step(opt_G)
                scaler_G.update()
            else:
                x, xp = G(z, gl)
                with torch.no_grad():
                    px, pxp = model(x), model(xp)
                t1 = -torch.mean((px - pxp) ** 2)
                t2 = l1 * torch.mean((x - xp) ** 2)
                t3 = l2 * (bce(D(x, gl), real) + bce(D(xp, gl), real)) / 2
                g_loss = t1 + t2 + t3
                g_loss.backward()
                opt_G.step()
            
            # TPU: Mark step for XLA optimization
            if TPU_AVAILABLE:
                xm.mark_step()
            
            # Discriminator
            opt_D.zero_grad(set_to_none=True)
            if use_amp_local:
                with torch.amp.autocast(device_type='cuda'):
                    x, xp = G(z, gl)
                    d_loss = (bce(D(imgs, labels), real) + bce(D(x.detach(), gl), fake_t) + bce(D(xp.detach(), gl), fake_t)) / 3
                scaler_D.scale(d_loss).backward()
                scaler_D.step(opt_D)
                scaler_D.update()
            else:
                x, xp = G(z, gl)
                d_loss = (bce(D(imgs, labels), real) + bce(D(x.detach(), gl), fake_t) + bce(D(xp.detach(), gl), fake_t)) / 3
                d_loss.backward()
                opt_D.step()
            
            # TPU: Mark step for XLA optimization
            if TPU_AVAILABLE:
                xm.mark_step()
    
    return G, D


@torch.no_grad()
def compute_bias(model, x, xp, device):
    """Compute bias as difference in model predictions between x and x'"""
    model.eval()
    x, xp = x.to(device), xp.to(device)
    
    if USE_AMP and not TPU_AVAILABLE:
        with torch.amp.autocast(device_type='cuda'):
            diff = torch.abs(model(x) - model(xp)).sum(1).mean()
    else:
        diff = torch.abs(model(x) - model(xp)).sum(1).mean()
    
    return diff.item()


def partition_data_non_iid_unequal(dataset, n_clients, alpha=0.5):
    """
    Create Non-IID partition with UNEQUAL data sizes per client.
    Uses Dirichlet distribution for label and size heterogeneity.
    """
    labels = np.array([dataset[i][1] for i in range(len(dataset))])
    n_classes = len(np.unique(labels))
    
    class_indices = {c: np.where(labels == c)[0] for c in range(n_classes)}
    for c in class_indices:
        np.random.shuffle(class_indices[c])
    
    client_indices = [[] for _ in range(n_clients)]
    
    for c in range(n_classes):
        proportions = np.random.dirichlet(np.repeat(alpha, n_clients))
        proportions = (proportions * len(class_indices[c])).astype(int)
        proportions[-1] = len(class_indices[c]) - proportions[:-1].sum()
        
        start = 0
        for client_id, num_samples in enumerate(proportions):
            if num_samples > 0:
                client_indices[client_id].extend(
                    class_indices[c][start:start + num_samples].tolist()
                )
            start += num_samples
    
    result = []
    for i in range(n_clients):
        indices = np.array(client_indices[i])
        np.random.shuffle(indices)
        result.append(indices)
    
    return result


@torch.no_grad()
def evaluate(model, loader, device):
    """Evaluate model accuracy (TPU/GPU compatible)"""
    model.eval()
    correct, total = 0, 0
    
    for d, t in loader:
        d, t = d.to(device), t.to(device)
        
        if USE_AMP and not TPU_AVAILABLE:
            with torch.amp.autocast(device_type='cuda'):
                preds = model(d).argmax(1)
        else:
            preds = model(d).argmax(1)
        
        correct += (preds == t).sum().item()
        total += len(t)
    
    return 100 * correct / total


@torch.no_grad()
def evaluate_per_client(model, client_loaders, device):
    """
    Evaluate model accuracy on EACH client's data.
    This measures how fairly the model performs across clients.
    """
    model.eval()
    client_accuracies = []
    
    for loader in client_loaders:
        correct, total = 0, 0
        for d, t in loader:
            d, t = d.to(device), t.to(device)
            
            if USE_AMP and not TPU_AVAILABLE:
                with torch.amp.autocast(device_type='cuda'):
                    preds = model(d).argmax(1)
            else:
                preds = model(d).argmax(1)
            
            correct += (preds == t).sum().item()
            total += len(t)
        
        acc = 100 * correct / total if total > 0 else 0
        client_accuracies.append(acc)
    
    return client_accuracies


# ============================================================
# FAIRNESS METRICS (Based on Per-Client Performance!)
# ============================================================

def calculate_jfi(performances):
    """Jain's Fairness Index: (Σpᵢ)² / (N × Σpᵢ²)"""
    p = np.array(performances)
    n = len(p)
    if np.sum(p ** 2) == 0:
        return 1.0
    return (np.sum(p) ** 2) / (n * np.sum(p ** 2))


def calculate_max_min_fairness(performances):
    """Max-Min Fairness: min(acc) / max(acc)"""
    p = np.array(performances)
    if np.max(p) == 0:
        return 0.0
    return np.min(p) / np.max(p)


def calculate_variance(performances):
    return np.var(performances)


def calculate_accuracy_gap(performances):
    return np.max(performances) - np.min(performances)


print("✅ Helper functions defined (TPU/GPU compatible)")

In [ ]:
# ============================================================
# CONFIGURATION
# ============================================================

# Training Parameters
N_ROUNDS = 50           # Total training rounds
N_CLIENTS = 20          # Number of federated clients
N_GAN_EPOCHS = 15       # GAN training epochs per round
N_PROBES = 500          # Number of probe samples
LOCAL_EPOCHS = 3        # Local training epochs per client

# ⭐ MULTI-GAMMA ABLATION STUDY
GAMMA_VALUES = [0.0, 0.3, 0.7]  # 0.0 = FedAvg baseline

# ⭐ Oscillation Fix Parameters
MOMENTUM = 0.8          # EMA momentum for fairness scores
WARMUP_ROUNDS = 5       # Rounds before activating fairness scoring
MU = 0.01               # FedProx proximal term strength

# DataLoader Parameters (TPU/GPU Optimized)
# TPU: batch size of 128 per core is optimal (128*8 = 1024 for TPU v3-8)
# But for federated learning with per-client training, we use smaller batches
if TPU_AVAILABLE:
    BATCH_SIZE = 128        # Per-client batch
    VAL_BATCH_SIZE = 512    # Larger for validation
    NUM_WORKERS = 4         # TPU can handle parallel loading
    PIN_MEMORY = False      # Not needed for TPU
    PREFETCH_FACTOR = 2
elif torch.cuda.is_available():
    BATCH_SIZE = 128
    VAL_BATCH_SIZE = 256
    NUM_WORKERS = 4
    PIN_MEMORY = True
    PREFETCH_FACTOR = 2
else:
    BATCH_SIZE = 64
    VAL_BATCH_SIZE = 128
    NUM_WORKERS = 0
    PIN_MEMORY = False
    PREFETCH_FACTOR = None

# Non-IID Parameters
DIRICHLET_ALPHA = 0.2  # Lower = more heterogeneous

print("=" * 60)
print("🔧 Fed-Audit-GAN v2.0 - CIFAR-10 ABLATION STUDY (TPU Edition)")
print("=" * 60)
print(f"Device: {DEVICE}")
print(f"TPU Mode: {TPU_AVAILABLE}")
print(f"Rounds: {N_ROUNDS}, Clients: {N_CLIENTS}")
print(f"\n🎯 GAMMA VALUES TO TEST: {GAMMA_VALUES}")
print(f"   γ=0.0 → FedAvg (NO GAN, data-weighted)")
print(f"   γ=0.3 → Mild fairness weighting")
print(f"   γ=0.7 → Strong fairness weighting")
print(f"\n⭐ OSCILLATION FIX PARAMETERS:")
print(f"   Momentum (β): {MOMENTUM}")
print(f"   Warm-up Rounds: {WARMUP_ROUNDS}")
print(f"   FedProx (μ): {MU}")
print(f"\n📦 BATCH SIZES:")
print(f"   Training: {BATCH_SIZE}")
print(f"   Validation: {VAL_BATCH_SIZE}")
print("=" * 60)

In [ ]:
# ============================================================
# DATA LOADING (CIFAR-10 with TPU/GPU Optimizations)
# ============================================================

# CIFAR-10 transforms with augmentation
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
])

train_data = datasets.CIFAR10('./data', train=True, download=True, transform=transform_train)
test_data = datasets.CIFAR10('./data', train=False, download=True, transform=transform_test)

# Create Non-IID partitions with UNEQUAL sizes
np.random.seed(42)
client_idx = partition_data_non_iid_unequal(train_data, N_CLIENTS, alpha=DIRICHLET_ALPHA)

# Calculate data weights for each client
client_data_sizes = [len(idx) for idx in client_idx]
total_samples = sum(client_data_sizes)
CLIENT_DATA_WEIGHTS = [size / total_samples for size in client_data_sizes]

# DataLoader kwargs (TPU compatible)
dataloader_kwargs = {
    'num_workers': NUM_WORKERS,
    'pin_memory': PIN_MEMORY,
}
# persistent_workers and prefetch_factor only for GPU/CPU
if not TPU_AVAILABLE and NUM_WORKERS > 0:
    dataloader_kwargs['persistent_workers'] = True
    if PREFETCH_FACTOR:
        dataloader_kwargs['prefetch_factor'] = PREFETCH_FACTOR

test_loader = DataLoader(test_data, batch_size=VAL_BATCH_SIZE, shuffle=False, **dataloader_kwargs)
val_loader = DataLoader(
    Subset(train_data, np.random.choice(len(train_data), 2000, replace=False)),
    batch_size=BATCH_SIZE, shuffle=False, **dataloader_kwargs
)

# Client data loaders
client_loaders = [
    DataLoader(Subset(train_data, client_idx[c]), batch_size=BATCH_SIZE, shuffle=True, **dataloader_kwargs)
    for c in range(N_CLIENTS)
]

print(f"✅ Data loaded!")
print(f"   Training samples: {len(train_data)}")
print(f"   Test samples: {len(test_data)}")
print(f"\n📊 NON-IID DATA DISTRIBUTION (Dirichlet α={DIRICHLET_ALPHA}):")
print(f"   Samples per client: {client_data_sizes}")
print(f"   Min: {min(client_data_sizes)}, Max: {max(client_data_sizes)}, Ratio: {max(client_data_sizes)/max(1, min(client_data_sizes)):.1f}x")
print(f"\n⚖️ CLIENT DATA WEIGHTS (for FedAvg):")
for i in range(min(5, N_CLIENTS)):  # Show first 5
    print(f"   Client {i}: {client_data_sizes[i]:5d} samples → weight = {CLIENT_DATA_WEIGHTS[i]:.4f}")
print(f"   ...")

In [ ]:
# ============================================================
# 🚀 MULTI-GAMMA ABLATION STUDY (4-PHASE ARCHITECTURE)
# FedAvg (γ=0) + Fed-Audit-GAN with γ=0.3, 0.7
# TPU/GPU Compatible
# ============================================================

def run_fed_audit_gan(gamma, n_rounds, n_clients, warmup_rounds, momentum, mu,
                      train_data, client_idx, val_loader, test_loader, client_loaders,
                      n_gan_epochs, n_probes, local_epochs, device, use_amp,
                      client_data_weights, tpu_available):
    """
    Run Fed-Audit-GAN v2.0 with specified gamma value.
    gamma=0 is FedAvg (data-weighted aggregation) - NO GAN training.
    TPU/GPU compatible.
    """
    
    # Initialize model
    model = CNN().to(device)
    
    # AMP scaler only for CUDA (not TPU)
    scaler = None
    if use_amp and not tpu_available:
        scaler = torch.amp.GradScaler(device='cuda')
    
    # Fairness score history for momentum
    fairness_history = {i: 0.0 for i in range(n_clients)}
    
    # History tracking
    history = {
        'acc': [], 'bias': [], 'alphas': [],
        'raw_scores': [], 'smoothed_scores': [],
        'client_accuracies': [],
        'jfi': [], 'max_min_fairness': [], 'variance': [], 'accuracy_gap': [],
        'min_client_acc': [], 'max_client_acc': []
    }
    
    is_fedavg = (gamma == 0)
    
    for rnd in tqdm(range(n_rounds), desc=f"{'FedAvg' if is_fedavg else f'γ={gamma}'}"):
        # ================================================================
        # PHASE 1: Local Client Training (with FedProx)
        # ================================================================
        updates = []
        global_params = [p.clone().detach() for p in model.parameters()]
        
        for cid in range(n_clients):
            local_model = copy.deepcopy(model)
            local_model.train()
            before_state = copy.deepcopy(model.state_dict())
            optimizer = optim.SGD(local_model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)
            
            for epoch in range(local_epochs):
                for data, target in client_loaders[cid]:
                    data = data.to(device)
                    target = target.to(device)
                    optimizer.zero_grad(set_to_none=True)
                    
                    if use_amp and not tpu_available:
                        with torch.amp.autocast(device_type='cuda'):
                            output = local_model(data)
                            ce_loss = F.cross_entropy(output, target)
                            prox_loss = sum(((lp - gp) ** 2).sum() 
                                          for lp, gp in zip(local_model.parameters(), global_params))
                            loss = ce_loss + (mu / 2) * prox_loss
                        scaler.scale(loss).backward()
                        scaler.step(optimizer)
                        scaler.update()
                    else:
                        output = local_model(data)
                        ce_loss = F.cross_entropy(output, target)
                        prox_loss = sum(((lp - gp) ** 2).sum() 
                                      for lp, gp in zip(local_model.parameters(), global_params))
                        loss = ce_loss + (mu / 2) * prox_loss
                        loss.backward()
                        optimizer.step()
                    
                    # TPU: Mark step for XLA optimization
                    if tpu_available:
                        xm.mark_step()
            
            update = {k: local_model.state_dict()[k] - before_state[k] for k in before_state}
            updates.append(update)
            del local_model
        
        # ================================================================
        # PHASE 2 & 3: GAN Training + Fairness Scoring (SKIP for FedAvg!)
        # ================================================================
        B_base = 0.0
        S_fair_raw = [0.0] * n_clients
        S_fair_smoothed = [0.0] * n_clients
        
        if not is_fedavg:
            G = FairnessGenerator(img_shape=(3, 32, 32)).to(device)
            D = Discriminator(img_shape=(3, 32, 32)).to(device)
            G, D = train_gan(G, D, model, val_loader, epochs=n_gan_epochs, device=device)
            
            G.eval()
            with torch.no_grad():
                z = torch.randn(n_probes, G.latent_dim, device=device)
                labels = torch.randint(0, 10, (n_probes,), device=device)
                
                if use_amp and not tpu_available:
                    with torch.amp.autocast(device_type='cuda'):
                        x_probe, xp_probe = G(z, labels)
                else:
                    x_probe, xp_probe = G(z, labels)
            
            B_base = compute_bias(model, x_probe, xp_probe, device)
            
            S_fair_raw = []
            S_fair_smoothed = []
            
            for cid, upd in enumerate(updates):
                hyp_model = copy.deepcopy(model)
                hyp_state = hyp_model.state_dict()
                for k in hyp_state:
                    hyp_state[k] = hyp_state[k] + upd[k]
                hyp_model.load_state_dict(hyp_state)
                
                B_client = compute_bias(hyp_model, x_probe, xp_probe, device)
                S_current = B_base - B_client
                S_fair_raw.append(S_current)
                
                S_prev = fairness_history[cid]
                S_smoothed = (momentum * S_prev) + ((1 - momentum) * S_current)
                fairness_history[cid] = S_smoothed
                S_fair_smoothed.append(S_smoothed)
                del hyp_model
            
            del G, D, x_probe, xp_probe
        
        history['raw_scores'].append(S_fair_raw.copy() if not is_fedavg else [0.0] * n_clients)
        history['smoothed_scores'].append(S_fair_smoothed.copy() if not is_fedavg else [0.0] * n_clients)
        
        # ================================================================
        # PHASE 4: Aggregation
        # ================================================================
        if is_fedavg:
            alphas = client_data_weights.copy()
        elif rnd < warmup_rounds:
            alphas = client_data_weights.copy()
        else:
            alphas = F.softmax(torch.tensor(S_fair_smoothed) * gamma, dim=0).tolist()
        
        new_state = model.state_dict()
        for k in new_state:
            new_state[k] = new_state[k] + sum(a * u[k] for a, u in zip(alphas, updates))
        model.load_state_dict(new_state)
        
        # TPU: Mark step after aggregation
        if tpu_available:
            xm.mark_step()
        
        # ================================================================
        # EVALUATION - Per-Client Fairness
        # ================================================================
        acc = evaluate(model, test_loader, device)
        client_accs = evaluate_per_client(model, client_loaders, device)
        
        jfi = calculate_jfi(client_accs)
        max_min = calculate_max_min_fairness(client_accs)
        var = calculate_variance(client_accs)
        gap = calculate_accuracy_gap(client_accs)
        
        history['acc'].append(acc)
        history['bias'].append(B_base)
        history['alphas'].append(alphas.copy())
        history['client_accuracies'].append(client_accs.copy())
        history['jfi'].append(jfi)
        history['max_min_fairness'].append(max_min)
        history['variance'].append(var)
        history['accuracy_gap'].append(gap)
        history['min_client_acc'].append(min(client_accs))
        history['max_client_acc'].append(max(client_accs))
        
        wandb.log({
            'round': rnd + 1,
            'accuracy': acc,
            'bias': B_base,
            'jfi': jfi,
            'max_min_fairness': max_min,
            'fairness_variance': var,
            'accuracy_gap': gap,
            'min_client_acc': min(client_accs),
            'max_client_acc': max(client_accs)
        })
    
    return model, history


# ============================================================
# RUN ALL EXPERIMENTS
# ============================================================

all_results = {}

for gamma in GAMMA_VALUES:
    method_name = "FedAvg" if gamma == 0 else f"FedAuditGAN_γ={gamma}"
    
    print(f"\n{'='*70}")
    print(f"🚀 RUNNING: {method_name}")
    if gamma == 0:
        print(f"   (Pure FedAvg - NO GAN, data-weighted aggregation)")
    else:
        print(f"   (Fed-Audit-GAN with fairness-aware aggregation)")
    print(f"   Device: {DEVICE} | TPU Mode: {TPU_AVAILABLE}")
    print(f"{'='*70}")
    
    wandb.init(
        project="FED_AUDIT_GAN_TEST_1_CIFAR10",
        name=f"{method_name}_CIFAR10_clients{N_CLIENTS}_TPU" if TPU_AVAILABLE else f"{method_name}_CIFAR10_clients{N_CLIENTS}",
        config={
            "method": method_name,
            "dataset": "CIFAR-10",
            "n_rounds": N_ROUNDS,
            "n_clients": N_CLIENTS,
            "gamma": gamma,
            "momentum": MOMENTUM,
            "warmup_rounds": WARMUP_ROUNDS,
            "mu_fedprox": MU,
            "dirichlet_alpha": DIRICHLET_ALPHA,
            "device": str(DEVICE),
            "tpu_enabled": TPU_AVAILABLE
        }
    )
    
    model, history = run_fed_audit_gan(
        gamma=gamma,
        n_rounds=N_ROUNDS,
        n_clients=N_CLIENTS,
        warmup_rounds=WARMUP_ROUNDS,
        momentum=MOMENTUM,
        mu=MU,
        train_data=train_data,
        client_idx=client_idx,
        val_loader=val_loader,
        test_loader=test_loader,
        client_loaders=client_loaders,
        n_gan_epochs=N_GAN_EPOCHS,
        n_probes=N_PROBES,
        local_epochs=LOCAL_EPOCHS,
        device=DEVICE,
        use_amp=USE_AMP,
        client_data_weights=CLIENT_DATA_WEIGHTS,
        tpu_available=TPU_AVAILABLE
    )
    
    wandb.finish()
    
    all_results[gamma] = {
        'model': model,
        'history': history,
        'name': method_name
    }
    
    print(f"✅ {method_name} Complete!")
    print(f"   Final Accuracy: {history['acc'][-1]:.2f}%")
    print(f"   Final JFI (per-client): {history['jfi'][-1]:.4f}")
    print(f"   Accuracy Gap: {history['accuracy_gap'][-1]:.2f}%")
    if gamma > 0:
        print(f"   Final Bias: {history['bias'][-1]:.6f}")

print("\n" + "=" * 70)
print("✅ ALL EXPERIMENTS COMPLETE!")
print("=" * 70)
print("📊 Check your WandB dashboard: https://wandb.ai")

In [ ]:
# ============================================================
# 📊 RESULTS SUMMARY TABLE
# ============================================================

print("\n" + "=" * 110)
print("📊 CIFAR-10 MULTI-GAMMA ABLATION STUDY RESULTS")
print("=" * 110)

print(f"\n{'METHOD':<25} {'GLOBAL ACC':<12} {'JFI':<10} {'MAX-MIN':<10} {'GAP':<10} {'MIN ACC':<10} {'MAX ACC':<10}")
print("-" * 110)

best_acc = max(all_results[g]['history']['acc'][-1] for g in GAMMA_VALUES)
best_jfi = max(all_results[g]['history']['jfi'][-1] for g in GAMMA_VALUES)
lowest_gap = min(all_results[g]['history']['accuracy_gap'][-1] for g in GAMMA_VALUES)

for gamma in GAMMA_VALUES:
    name = all_results[gamma]['name']
    acc = all_results[gamma]['history']['acc'][-1]
    jfi = all_results[gamma]['history']['jfi'][-1]
    max_min = all_results[gamma]['history']['max_min_fairness'][-1]
    gap = all_results[gamma]['history']['accuracy_gap'][-1]
    min_acc = all_results[gamma]['history']['min_client_acc'][-1]
    max_acc = all_results[gamma]['history']['max_client_acc'][-1]
    
    acc_mark = "🏆" if acc == best_acc else ""
    jfi_mark = "⭐" if jfi == best_jfi else ""
    gap_mark = "✅" if gap == lowest_gap else ""
    
    print(f"{name:<25} {acc:>8.2f}% {acc_mark:<2} {jfi:>8.4f} {jfi_mark:<2} {max_min:>8.4f}   {gap:>6.2f}% {gap_mark:<2} {min_acc:>8.2f}%  {max_acc:>8.2f}%")

print("=" * 110)

# Improvement over FedAvg
fedavg_acc = all_results[0.0]['history']['acc'][-1]
fedavg_jfi = all_results[0.0]['history']['jfi'][-1]
fedavg_gap = all_results[0.0]['history']['accuracy_gap'][-1]

print(f"\n📈 IMPROVEMENT OVER FedAvg:")
for gamma in GAMMA_VALUES:
    if gamma == 0:
        continue
    name = all_results[gamma]['name']
    acc = all_results[gamma]['history']['acc'][-1]
    jfi = all_results[gamma]['history']['jfi'][-1]
    gap = all_results[gamma]['history']['accuracy_gap'][-1]
    
    acc_diff = acc - fedavg_acc
    jfi_diff = jfi - fedavg_jfi
    gap_reduction = fedavg_gap - gap
    
    print(f"   {name}:")
    print(f"      Accuracy: {'+' if acc_diff >= 0 else ''}{acc_diff:.2f}%")
    print(f"      JFI: {'+' if jfi_diff >= 0 else ''}{jfi_diff:.4f}")
    print(f"      Gap Reduction: {gap_reduction:.2f}%")

In [ ]:
# ============================================================
# 📊 COMPREHENSIVE VISUALIZATION
# ============================================================

colors = {
    0.0: '#e74c3c',   # Red - FedAvg
    0.3: '#3498db',   # Blue
    0.7: '#2ecc71',   # Green
}

fig, axes = plt.subplots(2, 3, figsize=(18, 12))
rounds = range(1, N_ROUNDS + 1)

# Plot 1: Global Accuracy
ax = axes[0, 0]
for gamma in GAMMA_VALUES:
    name = all_results[gamma]['name']
    acc = all_results[gamma]['history']['acc']
    linestyle = '--' if gamma == 0 else '-'
    ax.plot(rounds, acc, color=colors[gamma], linestyle=linestyle, 
            linewidth=2, label=name)
ax.axvspan(1, WARMUP_ROUNDS, alpha=0.15, color='gray', label='Warm-up')
ax.set_xlabel('Round', fontsize=12)
ax.set_ylabel('Accuracy (%)', fontsize=12)
ax.set_title('CIFAR-10: Global Test Accuracy', fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

# Plot 2: JFI
ax = axes[0, 1]
for gamma in GAMMA_VALUES:
    name = all_results[gamma]['name']
    jfi = all_results[gamma]['history']['jfi']
    linestyle = '--' if gamma == 0 else '-'
    ax.plot(rounds, jfi, color=colors[gamma], linestyle=linestyle, linewidth=2, label=name)
ax.axvspan(1, WARMUP_ROUNDS, alpha=0.15, color='gray')
ax.set_xlabel('Round', fontsize=12)
ax.set_ylabel('JFI', fontsize=12)
ax.set_title("Jain's Fairness Index (Higher = Fairer)", fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

# Plot 3: Accuracy Gap
ax = axes[0, 2]
for gamma in GAMMA_VALUES:
    name = all_results[gamma]['name']
    gap = all_results[gamma]['history']['accuracy_gap']
    linestyle = '--' if gamma == 0 else '-'
    ax.plot(rounds, gap, color=colors[gamma], linestyle=linestyle, linewidth=2, label=name)
ax.axvspan(1, WARMUP_ROUNDS, alpha=0.15, color='gray')
ax.set_xlabel('Round', fontsize=12)
ax.set_ylabel('Accuracy Gap (%)', fontsize=12)
ax.set_title('Best-Worst Client Gap (Lower = Fairer)', fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

# Plot 4: Variance
ax = axes[1, 0]
for gamma in GAMMA_VALUES:
    name = all_results[gamma]['name']
    var = all_results[gamma]['history']['variance']
    linestyle = '--' if gamma == 0 else '-'
    ax.plot(rounds, var, color=colors[gamma], linestyle=linestyle, linewidth=2, label=name)
ax.axvspan(1, WARMUP_ROUNDS, alpha=0.15, color='gray')
ax.set_xlabel('Round', fontsize=12)
ax.set_ylabel('Variance', fontsize=12)
ax.set_title('Per-Client Accuracy Variance (Lower = Fairer)', fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

# Plot 5: Min-Max Range
ax = axes[1, 1]
for gamma in GAMMA_VALUES:
    name = all_results[gamma]['name']
    min_acc = all_results[gamma]['history']['min_client_acc']
    max_acc = all_results[gamma]['history']['max_client_acc']
    linestyle = '--' if gamma == 0 else '-'
    ax.fill_between(rounds, min_acc, max_acc, color=colors[gamma], alpha=0.2)
    ax.plot(rounds, min_acc, color=colors[gamma], linestyle=linestyle, linewidth=1.5)
    ax.plot(rounds, max_acc, color=colors[gamma], linestyle=linestyle, linewidth=1.5, label=name)
ax.set_xlabel('Round', fontsize=12)
ax.set_ylabel('Accuracy (%)', fontsize=12)
ax.set_title('Min-Max Client Accuracy Range', fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

# Plot 6: Final Per-Client Accuracy
ax = axes[1, 2]
x = np.arange(N_CLIENTS)
width = 0.25
for i, gamma in enumerate(GAMMA_VALUES):
    name = all_results[gamma]['name']
    client_accs = all_results[gamma]['history']['client_accuracies'][-1]
    ax.bar(x + i*width, client_accs, width, label=name, color=colors[gamma], alpha=0.8)
ax.set_xlabel('Client ID', fontsize=12)
ax.set_ylabel('Accuracy (%)', fontsize=12)
ax.set_title('Per-Client Accuracy (Final Round)', fontsize=14, fontweight='bold')
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('cifar10_fed_audit_gan_results.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n📁 Results saved to: cifar10_fed_audit_gan_results.png")

In [ ]:
# ============================================================
# SAVE ALL MODELS AND RESULTS (TPU Compatible)
# ============================================================

import os
import pickle

os.makedirs('results_cifar10_v2', exist_ok=True)

for gamma in GAMMA_VALUES:
    name = all_results[gamma]['name']
    filename = f"results_cifar10_v2/{name.replace('=', '').replace('.', '_')}_CIFAR10.pth"
    
    # Move model to CPU before saving (required for TPU)
    model_state = all_results[gamma]['model'].cpu().state_dict()
    
    # For TPU, use xm.save for proper serialization
    if TPU_AVAILABLE:
        import torch_xla.utils.serialization as xser
        xser.save({
            'model_state_dict': model_state,
            'history': all_results[gamma]['history'],
            'config': {
                'n_rounds': N_ROUNDS,
                'n_clients': N_CLIENTS,
                'gamma': gamma,
                'momentum': MOMENTUM,
                'warmup_rounds': WARMUP_ROUNDS,
                'mu': MU,
                'tpu_trained': True
            }
        }, filename)
    else:
        torch.save({
            'model_state_dict': model_state,
            'history': all_results[gamma]['history'],
            'config': {
                'n_rounds': N_ROUNDS,
                'n_clients': N_CLIENTS,
                'gamma': gamma,
                'momentum': MOMENTUM,
                'warmup_rounds': WARMUP_ROUNDS,
                'mu': MU,
                'tpu_trained': False
            }
        }, filename)
    print(f"✅ Saved: {filename}")

with open('results_cifar10_v2/all_results_summary.pkl', 'wb') as f:
    summary = {
        gamma: {
            'name': all_results[gamma]['name'],
            'history': all_results[gamma]['history'],
            'final_acc': all_results[gamma]['history']['acc'][-1],
            'final_jfi': all_results[gamma]['history']['jfi'][-1]
        }
        for gamma in GAMMA_VALUES
    }
    pickle.dump(summary, f)
print("✅ Saved: results_cifar10_v2/all_results_summary.pkl")

print(f"\n" + "=" * 60)
print("📊 FINAL RESULTS SUMMARY")
print("=" * 60)
for gamma in GAMMA_VALUES:
    name = all_results[gamma]['name']
    acc = all_results[gamma]['history']['acc'][-1]
    jfi = all_results[gamma]['history']['jfi'][-1]
    gap = all_results[gamma]['history']['accuracy_gap'][-1]
    print(f"   {name}: {acc:.2f}% accuracy, JFI={jfi:.4f}, Gap={gap:.2f}%")
print("=" * 60)
print(f"\n📊 Check WandB dashboard: https://wandb.ai")
print(f"   TPU Training: {TPU_AVAILABLE}")